## Initialising the Grid 
This part of the code aims to solve poisson's equation in the form $\nabla^{2}V=\rho\left ( x,y \right )$

Here we define a function that wil initialise a spatial grid, guess an initial solution for $V$ and define our source $\rho\left ( x,y \right )$

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, IntSlider,BoundedIntText
n=80
nt=20

def gridGen(n,initFunc,pFunc):
    vals = np.linspace(-5,5,n+1)
    x,y = np.meshgrid(vals,vals)
    initFuncs = {'Zero':np.zeros((n,n)),
             'Random':np.random.uniform(low=0, high=1, size=(n,n)),
             'x^2+y^2':x**2+y**2,
             'sin(x)^2':np.sin(x)**2,
             'sin(x)^2+sin(y)^2':np.sin(x)**2+np.sin(y)**2}
    grid = initFuncs[initFunc]
    gridNorm = np.ones((n,n))*0.5
    gridNorm[1:(n-1),1:(n-1)] = grid[1:(n-1),1:(n-1)]
    gridNorm = gridNorm/np.max(gridNorm)
    p = np.zeros((n,n))
    if pFunc == 'Shell Capacitor':
        for r in [int(n*.4),int(n*.35)]:
            x1 = np.arange(-r+1,r+1)
            y1 = ((r**2-x1**2)**.5).astype(int)
            x1 = (x1-n/2).astype(int)
            y1 = (y1-n/2).astype(int)
            p[[x1,y1,-x1,-y1],[y1,x1,-y1,-x1]]=1
    elif pFunc == 'Capacitor 1': 
        p[[int(n*.45),int(n*.55)],int(n*.1):int(n*.9)]=1
    elif pFunc == 'Capacitor 2': 
        p[[int(n*.25),int(n*.75)],int(n*.25):int(n*.75)]=1
    elif pFunc == 'Point Charge':
        p[int(n*.5),int(n*.5)]=1
                       
    return [gridNorm,x,y,p]

def pltt(n,Initialise,Source):
    [gridNorm,x,y,p] = gridGen(n,Initialise,Source)
    f, (ax1, ax2) = plt.subplots(1, 2,sharey=True,figsize=(8,4))
    im = ax1.pcolor(x,y,gridNorm,cmap='YlOrRd')
    im1 = ax2.pcolor(x,y,p,cmap='YlOrRd')
    ax1.set_title('Initial Solution')

    ax2.set_title('Source')
    ax2.set_aspect('equal')
    plt.show()
    
interact(pltt,
         n=widgets.IntSlider(min=20,max=150,continuous_update=False,description = 'Grid size'),
         Initialise=['Zero','Random','x^2+y^2','sin(x)^2+sin(y)^2'],
         Source=['Point Charge','Capacitor 1','Capacitor 2','Shell Capacitor'])



interactive(children=(IntSlider(value=20, continuous_update=False, description='Grid size', max=150, min=20), Dropdown(description='Initialise', options=('Zero', 'Random', 'x^2+y^2', 'sin(x)^2+sin(y)^2'), value='Zero'), Dropdown(description='Source', options=('Point Charge', 'Capacitor 1', 'Capacitor 2', 'Shell Capacitor'), value='Point Charge'), Output()), _dom_classes=('widget-interact',))

<function __main__.pltt>

We now introduce an iterating function that can be called throughout the code to iterate from a desired initial state, until either a fixed endpoint, or until a convergence condition is met

In [25]:
def iterate(timeStep,p,initialState,end=0,tol=2):
    state = np.copy(initialState)
    allStates= [initialState]
    dif=1
    t=0
    while dif > tol or t < end:
        state= timeStep(state,p)
        allStates.append(state)
        difM = abs(allStates[-1]/allStates[-2]-1)
        dif = np.amax(difM)
        t+=1

        
    return np.array(allStates)

Two classical timestep functions are now defined. The first applies to the Jacobi method and the second to the Gauss Seidel.

In [26]:
def JacobiStep(state,p):
    n = np.shape(state)[0]
    newState = np.copy(state)
    a1=state[1:n-1,2:]
    a2=state[1:n-1,0:n-2]
    a3=state[0:n-2,1:n-1]
    a4=state[2:,1:n-1]
    newState[1:n-1,1:n-1]=(a1+a2+a3+a4+p[1:n-1,1:n-1])/4
    return newState

def GaussSeidelStep(state,p):
    n = np.shape(state)[0]
    newState=np.copy(state)
    for i in range(1,n-1):
        for j in range(1,n-1):
            newState[i,j] = (newState[i-1,j]+newState[i+1,j]+newState[i,j-1]+newState[i,j+1]+p[i,j])/4
    return newState



In [39]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, IntSlider,BoundedIntText

def plot(states,t,contour=False):
    fig = plt.figure(figsize=(6,5))
    ax = fig.add_subplot(111)
    im = ax.contourf(x[:-1,:-1],y[:-1,:-1],states[t,:,:],100, cmap='YlOrRd')
    if contour == True:
        im1 = ax.contour(im,levels=im.levels[::5],colors='black',linewidths=0.5)
    ax.set_aspect('equal')
    fig.colorbar(im)
    plt.show()

def f(Tolerance,Iteration,gridNorm,Source):
    tol = Tolerance
    timeSteps = {'Jacobi Method':JacobiStep,'Gauss Seidel Method':GaussSeidelStep}
    timeStep = timeSteps[Iteration]
    gridNorm,x,y,p = gridGen(60,'Random',Source)    
    allStates = iterate(timeStep,p,gridNorm,tol=tol)
    nt=np.shape(allStates)[0]
    interact(plot,
             states=fixed(allStates),
             t=IntSlider(min=0,max=nt-1,continuous_update=False,description = 'Iteration #'),
             contour=widgets.Checkbox(description = 'Show Contours'))

interact(f,
         Tolerance=[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
         Iteration=['Jacobi Method', 'Gauss Seidel Method'],
         gridNorm=fixed(gridNorm),Source=['Point Charge','Capacitor 1','Capacitor 2','Shell Capacitor'])

interactive(children=(Dropdown(description='Tolerance', options=(0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06), value=0.1), Dropdown(description='Iteration', options=('Jacobi Method', 'Gauss Seidel Method'), value='Jacobi Method'), Dropdown(description='Source', options=('Point Charge', 'Capacitor 1', 'Capacitor 2', 'Shell Capacitor'), value='Point Charge'), Output()), _dom_classes=('widget-interact',))

<function __main__.f>